# Analyzing download speeds in Kentucky counties using Python

In [1]:
from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from shapely.geometry import Point
from adjustText import adjust_text
import requests

---

## Download data

First, download the data using the link below

In [2]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

In [10]:
tiles_in_state_counties

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,year,q,index_right,...,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,avg_d_mbps,avg_u_mbps,county
273,0320021220133221,253925,193275,16,79,18,"POLYGON ((-86.87439 34.75064, -86.86890 34.750...",2022,1,52702,...,NaN,NaN,S,143956,0,+34.7467037,-086.8669213,253.925,193.275,01083
631944,0320021220133230,291910,173785,17,33,16,"POLYGON ((-86.86890 34.75064, -86.86340 34.750...",2022,1,52702,...,NaN,NaN,S,143956,0,+34.7467037,-086.8669213,291.910,173.785,01083
2699952,0320021220133232,405245,294253,14,8,6,"POLYGON ((-86.86890 34.74613, -86.86340 34.746...",2022,1,52702,...,NaN,NaN,S,143956,0,+34.7467037,-086.8669213,405.245,294.253,01083
5847595,0320021220133223,346721,290856,15,4,1,"POLYGON ((-86.87439 34.74613, -86.86890 34.746...",2022,1,52702,...,NaN,NaN,S,143956,0,+34.7467037,-086.8669213,346.721,290.856,01083
273,0320021220133221,253925,193275,16,79,18,"POLYGON ((-86.87439 34.75064, -86.86890 34.750...",2022,1,12172,...,NaN,NaN,S,89028,0,+34.7464251,-086.8700647,253.925,193.275,01083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6742164,0320021232121303,63593,10920,25,2,1,"POLYGON ((-86.23718 34.50203, -86.23169 34.502...",2022,1,38125,...,NaN,NaN,S,22711,0,+34.4970827,-086.2305783,63.593,10.920,01095
6742164,0320021232121303,63593,10920,25,2,1,"POLYGON ((-86.23718 34.50203, -86.23169 34.502...",2022,1,10279,...,NaN,NaN,S,204165,0,+34.5004782,-086.2386854,63.593,10.920,01095
6742164,0320021232121303,63593,10920,25,2,1,"POLYGON ((-86.23718 34.50203, -86.23169 34.502...",2022,1,73036,...,NaN,NaN,S,991195,0,+34.5072867,-086.2333065,63.593,10.920,01095
6742260,0320021222301222,94442,11199,23,1,1,"POLYGON ((-86.96777 34.42050, -86.96228 34.420...",2022,1,72987,...,NaN,NaN,S,7371,0,+34.4161656,-086.9664945,94.442,11.199,01103


In [17]:
pdf

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
273,010830212022077,253.925,193.275,79,18,16,2022,1
631944,010830212022077,291.910,173.785,33,16,17,2022,1
2699952,010830212022077,405.245,294.253,8,6,14,2022,1
5847595,010830212022077,346.721,290.856,4,1,15,2022,1
273,010830212022076,253.925,193.275,79,18,16,2022,1
...,...,...,...,...,...,...,...,...
6695763,010830202023002,356.915,21.739,7,1,23,2022,1
6695763,010830202022065,356.915,21.739,7,1,23,2022,1
6706580,010830204041047,48.893,10.290,13,1,38,2022,1
6715217,010830202023007,13.453,1.068,7,1,24,2022,1


In [5]:
# For testing
year = 2022
t = "fixed"
q =1
tile_url = get_tile_url("fixed", year, q)
tiles = gp.read_file(tile_url)
tiles['year'] = year
tiles['q'] = q

In [10]:
tiles
county_url = 'https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_{state}_tabblock20.zip'
state = '01'
counties = gp.read_file(county_url.format(state=state))
state_counties = counties.to_crs(4326)

In [16]:

tiles_in_state_counties = gp.sjoin(tiles, state_counties, how="inner", predicate='intersects')
# convert to Mbps for easier reading
tiles_in_state_counties['avg_d_mbps'] = tiles_in_state_counties['avg_d_kbps'] / 1000
tiles_in_state_counties['avg_u_mbps'] = tiles_in_state_counties['avg_u_kbps'] / 1000
tiles_in_state_counties['county'] = tiles_in_state_counties['GEOID20'].apply(lambda x: x[:5])

county = tiles_in_state_counties['county'].unique()[1]
pdf = tiles_in_state_counties[tiles_in_state_counties['county'] == county]
pdf = pdf.reindex(columns = ['GEOID20', 'avg_d_mbps', 'avg_u_mbps', 'tests', 'devices', 'avg_lat_ms', 'year', 'q'])
export_path =  '../../data/{year}_{geoid}_{measure}.csv.xz'.format(year=year, geoid=county, measure='ookla')

pdf = pdf.rename(columns={'GEOID20': 'geoid'}) 
pdf

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
307,010010208032004,300.460,346.816,2,2,2,2022,1
509685,010010208032004,420.402,540.529,1,1,3,2022,1
2148267,010010208032004,453.762,644.528,4,1,3,2022,1
3020599,010010208032004,209.799,211.998,2,2,1,2022,1
4825061,010010208032004,456.482,550.670,3,1,2,2022,1
...,...,...,...,...,...,...,...,...
6494919,010010208033012,12.405,0.946,2,1,30,2022,1
6579887,010010203001000,280.201,21.949,2,1,28,2022,1
6605700,010010210002031,208.817,211.611,1,1,2,2022,1
6665010,010010210003018,34.837,7.225,2,1,51,2022,1


In [ ]:
value_vars = list(pdf.columns)
value_vars.remove("geoid")            
pdf = pd.melt(pdf, id_vars=["geoid", "year", "q"], value_name = 'value')      
# print(pdf)
# converting the data frame to a standard long format
pdf = pdf.reindex(expected_cols, axis=1)  

In [23]:
year = 2022
t = "fixed"
expected_cols = requests.get(
    "https://raw.githubusercontent.com/uva-bi-sdad/sdc.metadata/master/data/column_structure.json"
).json()

# For every quarter
pbar = tqdm(range(1,5))
for q in pbar:
    tile_url = get_tile_url("fixed", year, q)
    tiles = gp.read_file(tile_url)
    tiles['year'] = year
    tiles['q'] = q
    county_url = 'https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_{state}_tabblock20.zip'
    pbar.set_description('Tiles read')
    
    # For the states of interest (alabama, dc, georgia, maryland, virginia)
    for state in ['01', '11', '13', '24', '51']:
        pbar.set_description('Parsing for state: %s' % state)
        counties = gp.read_file(county_url.format(state=state))
        state_counties = counties.to_crs(4326)
        tiles_in_state_counties = gp.sjoin(tiles, state_counties, how="inner", predicate='intersects')
        # convert to Mbps for easier reading
        tiles_in_state_counties['avg_d_mbps'] = tiles_in_state_counties['avg_d_kbps'] / 1000
        tiles_in_state_counties['avg_u_mbps'] = tiles_in_state_counties['avg_u_kbps'] / 1000
        tiles_in_state_counties['county'] = tiles_in_state_counties['GEOID20'].apply(lambda x: x[:5])
        
        for county in tiles_in_state_counties['county'].unique():
            pbar.set_description('Processing county: %s' % county)
            pdf = tiles_in_state_counties[tiles_in_state_counties['county'] == county]
            pdf = pdf.reindex(columns = ['GEOID20', 'avg_d_mbps', 'avg_u_mbps', 'tests', 'devices', 'avg_lat_ms', 'year', 'q'])
            export_path =  '../../data/{year}_{geoid}_{measure}.csv.xz'.format(year=year, geoid=county, measure='ookla')
            
            pdf = pdf.rename(columns={'GEOID20': 'geoid'})
            value_vars = list(pdf.columns)
            value_vars.remove("geoid")            
            pdf = pd.melt(pdf, id_vars=["geoid", "year", "q"])      
            # print(pdf)
            # converting the data frame to a standard long format
            pdf = pdf.reindex(expected_cols, axis=1)        
            
            if os.path.isfile(export_path):
                edf = pd.read_csv(export_path, dtype={'geoid':object})
                pdf = pd.concat([pdf,edf])
                pdf = pdf.drop_duplicates()
            
            pbar.set_description('Saving to: %s' % export_path)
            pdf.to_csv(export_path, index = False)        

Saving to: ../../data/2022_51550_ookla.csv.xz: 100%|██████████| 4/4 [2:05:43<00:00, 1885.97s/it]  


---